In [1]:
from GmGM import GmGM, Dataset
import anndata as ad
import scipy.sparse as sparse
import scanpy as sc
import numpy as np
import dask.dataframe as dd
import dask.array as da

/Users/baileyandrew/mambaforge/envs/GmGM-python-demo/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


How to construct `data/adata.h5ad` (`data/adata_train.parquet` should come from composition website on Kaggle.)
```python
data = dd.read_parquet("../data/adata_train.parquet")
data["obs_id"] = data["obs_id"].astype("category").cat.as_known()
data["gene"] = data["gene"].astype("category").cat.as_known()
data["obs_code"] = data["obs_id"].cat.codes
data["gene_code"] = data["gene"].cat.codes
count_matrix = sparse.coo_array(
    (
        data["normalized_count"].compute().values,
        (data["obs_code"].compute().values, data["gene_code"].compute().values)
    )
)
adata = ad.AnnData(
    X=sparse.csr_matrix(count_matrix),
    obs=data["obs_id"].cat.categories.values.to_numpy(),
    var=data["gene"].cat.categories.values.to_numpy(),
)
adata.obs["obs_id"] = adata.obs[0]
adata.var["gene"] = adata.var[0]
del adata.obs[0]
del adata.var[0]
sc.pp.highly_variable_genes(
    adata,
    min_mean=0.0125,
    max_mean=3,
    min_disp=0.5
)
```

In [2]:
adata = ad.read_h5ad("../data/adata.h5ad")
adata

AnnData object with n_obs × n_vars = 240090 × 21255
    obs: 'obs_id'
    var: 'gene'

In [3]:
GmGM(
    adata,
    to_keep={
        "obs": 0,
        "var": 0
    },
    n_comps=20,
    verbose=True,
    readonly=False,
    # don't center so that data is not densified
    centering_method=None
)

Centering...
Calculating eigenvectors...
	by calculating SVD...
Calculating eigenvalues...
@0: inf - Line Search Gave Up!
Converged! (@0: inf)
Recomposing sparse precisions...


/Users/baileyandrew/mambaforge/envs/GmGM-python-demo/lib/python3.9/site-packages/GmGM/core/core.py:406: RuntimeWarning: invalid value encountered in scalar subtract
  err_diff: float = np.abs(prev_err - err)


Converting back to AnnData...


AnnData object with n_obs × n_vars = 240090 × 21255
    obs: 'obs_id'
    var: 'gene'
    uns: 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'var_gmgm_connectivities'

In [ ]:
# Reset...
del adata
adata = ad.read_h5ad("../data/adata.h5ad")
adata

# Time
print("Timing...")
%timeit -r 1 -n 1 GmGM(adata, to_keep={"obs": 0, "var": 0}, n_comps=20, readonly=False, centering_method=None)

/Users/baileyandrew/mambaforge/envs/GmGM-python-demo/lib/python3.9/site-packages/GmGM/core/core.py:406: RuntimeWarning: invalid value encountered in scalar subtract
  err_diff: float = np.abs(prev_err - err)


KeyboardInterrupt: 

In [3]:
# Reset...
del adata
adata = ad.read_h5ad("../data/adata.h5ad")
adata

# Time
print("Timing...")
%timeit -r 1 -n 1 GmGM(adata, to_keep={"obs": 0, "var": 0}, n_comps=20, readonly=False, centering_method=None, dont_recompose={"obs"})

Timing...


/Users/baileyandrew/mambaforge/envs/GmGM-python-demo/lib/python3.9/site-packages/GmGM/core/core.py:406: RuntimeWarning: invalid value encountered in scalar subtract
  err_diff: float = np.abs(prev_err - err)


KeyError: 'obs'